In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
import warnings

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.preprocessing import QuantileTransformer, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedShuffleSplit
from xgboost import XGBClassifier
import optuna

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
warnings.filterwarnings('ignore')

In [2]:
train=pd.read_csv('/kaggle/input/playground-series-s4e8/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv')
train['dataset'] = 'train'
test['dataset'] = 'test'


df_ = pd.concat([train, test], sort=False)
df=df_.copy()

In [3]:
df['class'] = df['class'].replace({'e': 0, 'p': 1 })

In [4]:
df.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,dataset
0,0,0.0,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a,train
1,1,1.0,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w,train
2,2,0.0,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w,train
3,3,0.0,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u,train
4,4,0.0,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a,train


In [5]:
for col in df.columns :
    print (col," : ",df[col].nunique())


id  :  5194909
class  :  2
cap-diameter  :  4145
cap-shape  :  108
cap-surface  :  114
cap-color  :  109
does-bruise-or-bleed  :  29
gill-attachment  :  117
gill-spacing  :  66
gill-color  :  86
stem-height  :  2858
stem-width  :  6194
stem-root  :  45
stem-surface  :  87
stem-color  :  88
veil-type  :  24
veil-color  :  27
has-ring  :  26
ring-type  :  47
spore-print-color  :  43
habitat  :  65
season  :  4
dataset  :  2


In [6]:
for col_name in df.columns :
     print(pd.DataFrame({col_name: df[col_name].value_counts(),"Ratio": 100 * df[col_name].value_counts() / len(df)}))
     print("##########################################")

         id     Ratio
id                   
0         1  0.000019
3463271   1  0.000019
3463278   1  0.000019
3463277   1  0.000019
3463276   1  0.000019
...      ..       ...
1731635   1  0.000019
1731634   1  0.000019
1731633   1  0.000019
1731632   1  0.000019
5194908   1  0.000019

[5194909 rows x 2 columns]
##########################################
         class      Ratio
class                    
1.0    1705396  32.828217
0.0    1411549  27.171775
##########################################
              cap-diameter     Ratio
cap-diameter                        
1.49                 13638  0.262526
3.18                 13116  0.252478
3.14                 12317  0.237098
1.51                 11792  0.226991
3.28                 11403  0.219503
...                    ...       ...
30.81                    1  0.000019
44.48                    1  0.000019
32.50                    1  0.000019
38.29                    1  0.000019
58.98                    1  0.000019

[4145 rows x 2

In [7]:
def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]

    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")

    if na_name:
        return na_columns


missing_values_table(df)

                       n_miss  ratio
veil-type             4929038  94.88
spore-print-color     4749299  91.42
stem-root             4595035  88.45
veil-color            4567071  87.91
stem-surface          3302349  63.57
gill-spacing          2098030  40.39
class                 2077964  40.00
cap-surface           1117927  21.52
gill-attachment        873757  16.82
ring-type              215075   4.14
gill-color                106   0.00
cap-shape                  71   0.00
habitat                    70   0.00
stem-color                 59   0.00
has-ring                   43   0.00
cap-color                  25   0.00
does-bruise-or-bleed       18   0.00
cap-diameter               11   0.00
stem-height                 1   0.00


In [8]:
columns_to_drop = ['veil-type', 'spore-print-color', 'stem-root', 'veil-color']
df = df.drop(columns=columns_to_drop)

In [9]:
def grab_col_names(dataframe,cat_th=10,car_th=30):
    cat_cols=[col for col in dataframe.columns if dataframe[col].dtypes=='object' and col!='dataset']
    num_but_cat=[col for col in dataframe.columns if dataframe[col].dtypes!='O' and dataframe[col].nunique() < cat_th]
    cat_but_car=[col for col in dataframe.columns if dataframe[col].dtypes=='O' and dataframe[col].nunique()>car_th and col!='dataset']
    cat_cols=cat_cols+num_but_cat
    num_cols=[col for col in dataframe.columns if dataframe[col].dtypes!='O']
    num_cols=[col for col in num_cols if col not in num_but_cat]
    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 5194909
Variables: 19
cat_cols: 14
num_cols: 4
cat_but_car: 10
num_but_cat: 1


In [10]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)
cat_cols = [col for col in cat_cols if col not in ["class",'dataset']]
num_cols = [col for col in num_cols if col not in ["id"]]
print(cat_cols)

Observations: 5194909
Variables: 19
cat_cols: 14
num_cols: 4
cat_but_car: 10
num_but_cat: 1
['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-surface', 'stem-color', 'has-ring', 'ring-type', 'habitat', 'season']


In [11]:
numeric_imputer = SimpleImputer(strategy='median')
df[num_cols] = numeric_imputer.fit_transform(df[num_cols])

In [12]:
 for col in cat_cols:
        df[col] = df[col].fillna('missing')
        df.loc[df[col].value_counts(dropna=False)[df[col]].values < 100, col] = "noise"
        df[col] = df[col].astype('category')

In [13]:
#def replace_non_alpha_with_nan(df):
 #   object_columns = [col for col in df.columns if df[col].dtype == 'object' and col!='dataset']
 #   alphabet_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
  #  def filter_alpha(value):
  #      if isinstance(value, str):
  #          return value if len(value) == 1 and value in alphabet_list else np.nan
   #     return np.nan
    #for col in object_columns:
     #   df[col] = df[col].apply(filter_alpha)
    #return df

In [14]:
#df = replace_non_alpha_with_nan(df)

# Modeling : 

In [15]:
train_processed = df[df['dataset'] == 'train'].drop(columns=['dataset'])
test_processed = df[df['dataset'] == 'test'].drop(columns=['dataset'])



X = train_processed.drop(columns=['class'])
y = train_processed['class'] 


test_processed=test_processed.drop(columns=['class'])

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
from sklearn.metrics import accuracy_score
#model
def objective(trial):
    # Suggest hyperparameters for tuning
    param = {
        'objective': 'binary',  # Binary classification
        'metric': 'binary_logloss',  # Evaluation metric
        'boosting_type': 'gbdt',  # Type of boosting
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),  # L1 regularization
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0)   # L2 regularization
    }

    # Create dataset for LightGBM
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Train the model
    model = lgb.train(param, train_data, valid_sets=[valid_data],callbacks=[
        lgb.early_stopping(stopping_rounds=3), lgb.log_evaluation(50)]
    )
    
    # Make predictions
    preds = model.predict(X_valid)
    preds = [1 if p >= 0.5 else 0 for p in preds]
    
    # Calculate accuracy
    accuracy = accuracy_score(y_valid, preds)
    
    # Return accuracy as the objective value to maximize
    return accuracy


study = optuna.create_study(direction='maximize')  # 'maximize' because we want to maximize accuracy

# Start optimization
study.optimize(objective, n_trials=100)

# Print the best trial's parameters
print(f'Best trial parameters: {study.best_trial.params}')
print(f'Best accuracy: {study.best_value:.2f}')

best_params = study.best_trial.params

# Train the final model using the best parameters
final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(X_train, y_train)

# Evaluate on the test set
y_pred = final_model.predict(X_valid)
final_accuracy = accuracy_score(y_valid, y_pred)
print(f'Final accuracy on validation set: {final_accuracy * 100:.2f}%')

[I 2024-08-18 21:20:54,787] A new study created in memory with name: no-name-8e5f5a0d-4e9d-4d5d-b448-aae6652aa69f


[LightGBM] [Info] Number of positive: 1193768, number of negative: 988093
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.485063 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1141
[LightGBM] [Info] Number of data points in the train set: 2181861, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547133 -> initscore=0.189093
[LightGBM] [Info] Start training from score 0.189093
Training until validation scores don't improve for 3 rounds
[50]	valid_0's binary_logloss: 0.330793
[100]	valid_0's binary_logloss: 0.184285
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.184285


[I 2024-08-18 21:21:39,076] Trial 0 finished with value: 0.9889517947050747 and parameters: {'learning_rate': 0.013386317965741973, 'num_leaves': 115, 'max_depth': 13, 'feature_fraction': 0.6976567346375632, 'bagging_fraction': 0.7422183008310783, 'bagging_freq': 1, 'min_data_in_leaf': 94, 'lambda_l1': 7.052834714933471e-05, 'lambda_l2': 2.026833622530663}. Best is trial 0 with value: 0.9889517947050747.


[LightGBM] [Info] Number of positive: 1193768, number of negative: 988093
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.184620 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1141
[LightGBM] [Info] Number of data points in the train set: 2181861, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547133 -> initscore=0.189093
[LightGBM] [Info] Start training from score 0.189093
Training until validation scores don't improve for 3 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Lig

[I 2024-08-18 21:22:10,536] Trial 1 finished with value: 0.9708432611401756 and parameters: {'learning_rate': 0.040857009568322965, 'num_leaves': 35, 'max_depth': 5, 'feature_fraction': 0.7722633781647056, 'bagging_fraction': 0.6887938440382385, 'bagging_freq': 3, 'min_data_in_leaf': 54, 'lambda_l1': 5.825444740773138, 'lambda_l2': 2.0403510856616137e-08}. Best is trial 0 with value: 0.9889517947050747.


...

[LightGBM] [Info] Number of positive: 1193768, number of negative: 988093
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.168399 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1141
[LightGBM] [Info] Number of data points in the train set: 2181861, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547133 -> initscore=0.189093
[LightGBM] [Info] Start training from score 0.189093
Training until validation scores don't improve for 3 rounds
[50]	valid_0's binary_logloss: 0.0410837
[100]	valid_0's binary_logloss: 0.0379016
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.0379016


[I 2024-08-18 22:35:48,556] Trial 98 finished with value: 0.9916456703354992 and parameters: {'learning_rate': 0.11121207680020556, 'num_leaves': 148, 'max_depth': 20, 'feature_fraction': 0.9326309250420505, 'bagging_fraction': 0.8625568886110985, 'bagging_freq': 2, 'min_data_in_leaf': 12, 'lambda_l1': 6.013324617782155, 'lambda_l2': 0.004901922576257384}. Best is trial 58 with value: 0.9919675665501709.


[LightGBM] [Info] Number of positive: 1193768, number of negative: 988093
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.413607 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1141
[LightGBM] [Info] Number of data points in the train set: 2181861, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547133 -> initscore=0.189093
[LightGBM] [Info] Start training from score 0.189093
Training until validation scores don't improve for 3 rounds
[50]	valid_0's binary_logloss: 0.0381206
Early stopping, best iteration is:
[88]	valid_0's binary_logloss: 0.0375522


[I 2024-08-18 22:36:26,683] Trial 99 finished with value: 0.9918499300597593 and parameters: {'learning_rate': 0.17269567382126966, 'num_leaves': 132, 'max_depth': 18, 'feature_fraction': 0.6542568137621558, 'bagging_fraction': 0.92654232989455, 'bagging_freq': 1, 'min_data_in_leaf': 20, 'lambda_l1': 1.0144985510309017, 'lambda_l2': 0.07023896507863077}. Best is trial 58 with value: 0.9919675665501709.


Best trial parameters: {'learning_rate': 0.1570089046776225, 'num_leaves': 136, 'max_depth': 19, 'feature_fraction': 0.622981192126568, 'bagging_fraction': 0.8970254429615686, 'bagging_freq': 1, 'min_data_in_leaf': 10, 'lambda_l1': 2.6912461632162645, 'lambda_l2': 0.015406264409918633}
Best accuracy: 0.99
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.622981192126568, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.622981192126568
[LightGBM] [Warning] lambda_l2 is set=0.015406264409918633, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.015406264409918633
[LightGBM] [Warning] lambda_l1 is set=2.6912461632162645, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.6912461632162645
[LightGBM] [Warning] bagging_fraction is set=0.8970254429615686, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8970254429615686
[

In [18]:
y_final = final_model.predict(test_processed)

[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.622981192126568, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.622981192126568
[LightGBM] [Warning] lambda_l2 is set=0.015406264409918633, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.015406264409918633
[LightGBM] [Warning] lambda_l1 is set=2.6912461632162645, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.6912461632162645
[LightGBM] [Warning] bagging_fraction is set=0.8970254429615686, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8970254429615686
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


In [19]:
y_final

array([0., 1., 1., ..., 1., 0., 0.])

In [20]:
y_final_replaced = np.where(y_final == 0, 'e', 'p')

In [21]:
y_final_replaced

array(['e', 'p', 'p', ..., 'p', 'e', 'e'], dtype='<U1')

In [22]:
submission1 = pd.read_csv(r'/kaggle/input/playground-series-s4e8/sample_submission.csv')
submission1['class'] = y_final_replaced

In [23]:
submission1.to_csv('submission.csv', index=False)